In [1]:
from glob import glob

In [4]:
def print_stats(tuning_method, extract_method):
    print(f'Number of attentions from {tuning_method}', len(glob(f'outputs/{tuning_method}/attentions/batch_*')))
    print(f'Number of rationales using {extract_method}', len(glob(f'kr4/{tuning_method}_{extract_method}/batch_*')))
    print('Number of tokenized rationales', len(glob(f'kr4_tokenized/{tuning_method}_{extract_method}/batch_*')))

In [5]:
print_stats('FT3', 'TopK') # same for TopK-Un

Number of attentions from FT3 14345
Number of rationales using TopK 14171
Number of tokenized rationales 14171


In [6]:
print_stats('FT3', 'Contin') # same for Contin-Un

Number of attentions from FT3 14345
Number of rationales using Contin 11967
Number of tokenized rationales 11967


In [8]:
print_stats('FT3', 'RandomTopK')

Number of attentions from FT3 14345
Number of rationales using RandomTopK 14171
Number of tokenized rationales 14171


In [7]:
print_stats('FT3', 'RandomContin')

Number of attentions from FT3 14345
Number of rationales using RandomContin 14171
Number of tokenized rationales 14171


## See rationales

In [6]:
from datasets import load_from_disk, load_dataset

In [7]:
# original dataset
kr3 = load_dataset("Wittgensteinian/KR3", name='kr3', split='train')
kr3 = kr3.remove_columns(['__index_level_0__'])
kr3_binary = kr3.filter(lambda example: example['Rating'] != 2)

Using custom data configuration Wittgensteinian--KR3-0f424e029f3cf821
Reusing dataset parquet (/home/gsdsaml/.cache/huggingface/datasets/parquet/Wittgensteinian--KR3-0f424e029f3cf821/0.0.0/0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901)
Loading cached processed dataset at /home/gsdsaml/.cache/huggingface/datasets/parquet/Wittgensteinian--KR3-0f424e029f3cf821/0.0.0/0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901/cache-88ae1594470bd0c3.arrow


In [76]:
import numpy as np
for i in np.random.choice(14345,40):
    try:
        if len(kr3_binary[int(i)]['Review']) > 200:
            print(i)
    except KeyError:
        continue

9435
9860
7290
2706
719


In [91]:
# index
# guidelines below (* for label=0)
# interesting: 123
# long text: 8509, 2997, 1134, 6702, 9435*, 9860, 7290, 2706, 719
i = 719

In [92]:
kr3_binary[i]

{'Rating': 1,
 'Review': ' 이름  낮에는 가정 초밥이라는 밥집이고, 밤에는 토라 토라고 하는 술집이에요.  맛  고기 먹고 간단하게 한 잔 더 하고 싶어서 2차로 갔어요. 생맥주는 시원하니 맛있었고 야무네도 오랜만에 먹으니까 괜찮더라고요. 다만 명란 치즈 감자는 전체적으로 많이 짰어요. 배가 불러서 간단한 메뉴 시키고 싶었는데 나베나 초밥 같은 무거운 메뉴가 많아서 좀 아쉬웠어요.  분위기  일본 상점가 레스토랑 생각하시면 돼요. 전체적인 인테리어나 조명이나 소품 같은 것들이 분위기가 비슷해요. 오션뷰라 창가 쪽에 앉으시면 좀 더 시원한 느낌 받으실 수 있어요.'}

In [93]:
rationales = load_from_disk(f'kr4/FT3_TopK/batch_{i//32}')
rationales[i%32]

{'Rating': 1,
 'Rationale': '초밥이라는 간단하게 갔어요. 시원하니 맛있었고 분위기 레스토랑 생각하시면 돼요. 인테리어나 조명이나 것들이 분위기가 비슷해요. 오션뷰라 시원한',
 'Unrationale': '이름 낮에는 가정 밥집이고, 밤에는 토라 토라고 하는 술집이에요. 맛 고기 먹고 한 잔 더 하고 싶어서 2차로 생맥주는 야무네도 오랜만에 먹으니까 괜찮더라고요. 다만 명란 치즈 감자는 전체적으로 많이 짰어요. 배가 불러서 간단한 메뉴 시키고 싶었는데 나베나 초밥 같은 무거운 메뉴가 많아서 좀 아쉬웠어요. 일본 상점가 전체적인 소품 같은 창가 쪽에 앉으시면 좀 더 느낌 받으실 수 있어요.'}

In [94]:
rationales = load_from_disk(f'kr4/FT3_Contin/batch_{i//32}')
rationales[i%32]

{'Rating': 1,
 'Rationale': '분위기  일본 상점가 레스토랑 생각하시면 돼요. 전체적인 인테리어나 조명이나 소품 같은 것들이 분위기가 비슷해요. 오션뷰라',
 'Unrationale': ' 이름  낮에는 가정 초밥이라는 밥집이고, 밤에는 토라 토라고 하는 술집이에요.  맛  고기 먹고 간단하게 한 잔 더 하고 싶어서 2차로 갔어요. 생맥주는 시원하니 맛있었고 야무네도 오랜만에 먹으니까 괜찮더라고요. 다만 명란 치즈 감자는 전체적으로 많이 짰어요. 배가 불러서 간단한 메뉴 시키고 싶었는데 나베나 초밥 같은 무거운 메뉴가 많아서 좀 아쉬웠어요.  창가 쪽에 앉으시면 좀 더 시원한 느낌 받으실 수 있어요.'}